In [1]:
from dask.distributed import Client
from dask import delayed

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client("localhost:8000")

In [2]:
import dask.dataframe as dd
import pandas as pd


df = pd.read_csv("data/data.csv")
df.head()

,SYM Im adding more every month up till the point this hits 50 SYM after that I ll start working on my SYM SYM and SYM,Bullish
0,SYM SYM SYM SYM SYM FEDs balance sheet total a...,Bearish
1,SYM SYM SYM SYM SYM X,Bullish
2,SYM going down to 300 in next 2 weeks,Bearish
3,How will this affect the market SYM price may ...,Bullish
4,SYM pos,Bullish


In [3]:
len(df)

17530

In [4]:
# -*- coding: utf-8 -*-
import requests
import json
from json import JSONDecodeError
import re
import datetime
from datetime import date, timedelta
import time
import sys

def printToFile(linesOfData):
    with open('data/data.csv','a') as file:
        for line in linesOfData:
            file.write(line)
            file.write('\n')

def clean_tweet(tweet): 
    tweet = tweet.replace('\n',' ').replace('\r',' ')
    tweet = re.sub("\$[A-Z]*","<SYM>", tweet)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    
@delayed
def scrape(symbol, num_tweets=100, print_tweets=False):
    linesOfData = []
    tweets = []
    labels = []
    i = 0
    bulls = 0
    bears = 0
    page = 1
    totalTweets = 0
    
    ''' getting first page via IP rotation
        headers = {"apikey": "bf892dc0-ba87-11ea-aa7d-4f92d4ba4d88"}
        params = (("url","https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json"),);
        r = requests.get('https://app.zenscrape.com/api/v1/get', headers=headers, params=params);
    '''
    
    # getting first page without ip rotation.
    r = requests.get('https://api.stocktwits.com/api/2/streams/symbol/'+symbol+".json");

    jsonText = json.dumps(r.json())
    dictionary = json.loads(jsonText)
    maxi = dictionary['cursor']['max']
    length = len(dictionary['messages'])

    # number of tweets on the first page. 
    # if 30 then it works well
    #print(length)

    # master while loop can be configured 
    # either total labelled tweets or totalpages to scrape
    
    while totalTweets < 100: 
            
        while i < length:
            
            isBull = 0; 
            isBear = 0; 
            isNull = 0; 
            
            message = dictionary['messages'][i]
            
            x = message['body'].replace("&#39;", "'").replace("&quot;","")
            
            x = clean_tweet(x)
            
            # dealing with the time the comment was posted. 
            strTime = message['created_at'].replace("T"," ").replace("Z","")
            dateTime = datetime.datetime.strptime(strTime, '%Y-%m-%d %H:%M:%S')
        
           
            if(str(dictionary['messages'][i]['entities']['sentiment']) == "None"):
                isNull = 0; 
        
            else:
                if str(dictionary['messages'][i]['entities']['sentiment']['basic']) == "Bullish":
                        bulls += 1
                        isBull = 1
                        totalTweets += 1
                                            
                if str(dictionary['messages'][i]['entities']['sentiment']['basic']) == "Bearish":
                        bears += 1        
                        isBear = 1 
                        totalTweets += 1
        
        
            if(isBull and len(x) > 0):
                if print_tweets:
                    print(" ---------- "+str(totalTweets)+" | Page # "+str(page)+" | Tweet # "+str(i)+" | "+strTime+" | "+message['user']['username']+" | Bullish ---------- ")
                    print(x+"\n\n")
                linesOfData.append(x+",Bullish")
                tweets.append(x)
                labels.append("Bullish")
                
            if(isBear and len(x) > 0):
                if print_tweets:
                    print(" ---------- "+str(totalTweets)+" | Page # "+str(page)+" | Tweet # "+str(i)+" | "+strTime+" | "+message['user']['username']+" | Bearish ----------")
                    print(x+"\n\n")
                linesOfData.append(x+",Bearish")
                tweets.append(x)
                labels.append("Bearish")
                
            i += 1
        
        suc = 0
        while suc < 1:
            try:
                
                # wait time for call in seconds. 
                time.sleep(0.5)
                
                ''' using the IP rotating proxy 
                headers = {"apikey": "bf892dc0-ba87-11ea-aa7d-4f92d4ba4d88"}
                params = (("url","https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json?max="+str(maxi)),);
                r = requests.get('https://app.zenscrape.com/api/v1/get', headers=headers, params=params);
                '''
                
                # call without ip rotation. 
                r = requests.get("https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json?max="+str(maxi));
                
                
                jsonText = json.dumps(r.json())
                
                dictionary = json.loads(jsonText)
                
                maxi = dictionary['cursor']['max']
                
                length = len(dictionary['messages'])
                
                i = 0
                
                suc = 1
                
            except JSONDecodeError:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)

            except KeyError:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)
                                        
            except:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)
            
        page += 1
    
    #print("Printing to File")
    #printToFile(linesOfData)
    #print("Completed.")
    return pd.DataFrame(list(zip(tweets, labels)), columns=["tweets", "labels"])


@delayed
def mergeNewData(dfs=[]):
    if len(dfs) > 0:
        df1 = dfs[0]
        
        for df in dfs[0:]:
            df1 = df1.append(df, ignore_index=True)
        
        return df1

In [5]:
# test
@delayed
def inc(x):
    return x + 1

@delayed
def add(nums):
    total = 0
    for n in nums:
        total += n
    return total

In [6]:
# example with test code
x = inc(15)
y = inc(40)
z = inc(75)

nums = [x,y,z]
total = add(nums)
#total.visualize()
total.compute()

133

In [7]:
symbols = pd.read_csv("symbols.csv")
symbols = symbols["Symbol"]
for sym in symbols:
    print(sym)

MMM
AOS
ABT
ABBV
ACN
ATVI
AYI
ADBE
AAP
AMD
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANDV
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BHGE
BLL
BAC
BAX
BBT
BDX
BRK.B
BBY
BIIB
BLK
HRB
BA
BKNG
BWA
BXP
BSX
BHF
BMY
AVGO
BF.B
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CBRE
CBS
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DOV
DWDP
DPS
DTE
DUK
DRE
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
RE
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HRS
HIG
HAS
HCA
HCP
HP
HSIC
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HPQ
HUM
HBAN
HII
ID

In [8]:
# distributed scraping bitchhhh
amd = scrape("AMD", num_tweets=30)
nvda = scrape("NVDA", num_tweets=30)

#dfs = [scrape(sym, num_tweets=30) for sym in symbols]
dfs = [amd, nvda]
work = mergeNewData(dfs)
new_df = work.compute()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)